In [99]:
from mpi4py import MPI
from manapy.ast import Variable
from manapy.base.base import Struct
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.solvers.advec.tools_utils import initialisation_gaussian_2d
import numpy as np
import time
from numba import cuda

import matplotlib.pyplot as plt

from timeit import default_timer as timer
from manapy.cuda.utils import VarClass
from manapy.cuda.utils import GPU_Backend

###############
# test_time
###############

def test_time(iter, fun):
  #fun()
  start_time = timer()
  for _ in range(iter):
    fun()
  end_time = timer()
  elapsed_time = (end_time - start_time) / iter
  print(f"{elapsed_time * 1000:.5f} ms")
  #print(f"{elapsed_time * 1000000:.5f} micros")

###############
# init
###############
def init(dim, mesh_path):
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision="single")
  MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

  running_conf = Struct(backend="numba", signature=True, cache =True, float_precision="single")
  domain = Domain(dim=dim, conf=running_conf)
  ne = Variable(domain=domain)
  u  = Variable(domain=domain)
  v  = Variable(domain=domain)
  w  = Variable(domain=domain)
  
  P = Variable(domain=domain)
  Pinit = 2.0
  cells = domain.cells
  initialisation_gaussian_2d(ne.cell, u.cell, v.cell, P.cell, cells.center, Pinit)

  u.face[:] = 2.
  v.face[:] = 0.
  w.face[:] = 0.
  
  u.interpolate_facetocell()
  v.interpolate_facetocell()
  w.interpolate_facetocell()
  return (domain, ne, u, v, w, P)

In [100]:
GPU_Backend.init_stream()
dim = 3
#mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/manapy/mesh/2D/carre.msh"
mesh_file = "/home/aben-ham/Desktop/work/stage/manapy/mesh/3D/cube_bis.msh"
#mesh_file = "/home/ayoub.hamou/mesh/square.msh"
#mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/gpu_accelerator/functions/square.msh"
domain, ne, u, v, w, P = init(dim=dim, mesh_path=mesh_file)

I am the rank 0 i'm using this GPU b'NVIDIA GeForce GTX 950M' with id 0
Reading gmsh file ...
Saving partition files ...
Number of Cells: 4006
Number of Vertices: 1721
Local domain contruction ...


In [102]:
# domain._cells._tc
domain._cells._nodeid[0]
# domain._nodes._vertex
# domain._halos._halosint
# domain._halos._halosext
# domain._halos._centvol
# domain._cells._globtoloc
# domain._cells._loctoglob
# domain._nodes._loctoglob
# domain._halos._neigh
# domain._nodes._nparts

array([1453, 1482, 1422, 1564,    0,    0,    0,    0,    4], dtype=int32)

In [ ]:
print(domain._nbcells)
print(domain._cells._nodeid.shape)

In [ ]:
domain._typeOfCells

In [ ]:
# args list
#? The order is imported of the argument list

VarClass.convert_to_var_class([
    domain.nodes,
    domain.faces,
    domain.cells,
    domain.halos,
])

args = [
  domain._cells._faceid,
  domain._nbcells,
  domain._nbfaces,
  domain._faces._cellid, 
  domain._maxcellfid
]



In [ ]:
def create_cellsOfFace(faceid:'int32[:,:]', nbelements:'int32', nbfaces:'int32', cellid:'int32[:,:]', maxcellfid:'int32'):
    for i in range(nbelements):
        for j in range(faceid[i][-1]):
            if cellid[faceid[i][j]][0] == -1 :
              cellid[faceid[i][j]][0] = i
            else:
              cellid[faceid[i][j]][1] = i



In [ ]:
res = domain._faces._cellid

In [ ]:
res

In [ ]:
domain._faces._cellid

In [ ]:
domain._faces._cellid == res

In [ ]:
# def find_duplicates(arr):
#     unique_elements, counts = np.unique(arr, return_counts=True)
#     duplicates = unique_elements[counts > 1]
#     return duplicates

# #? There are duplicate elements on host_cellfid.
# #? Data race; use atomic operations to prevent it.
# # host_cellfid
# find_duplicates(domain._cells._faceid.ravel())

In [ ]:
arr = domain._cells._faceid
for i in range(len(arr)):
  if find_duplicates(arr[i][:-1]).shape[0] != 0:
    print(arr[i])

In [ ]:
from manapy.ddm.ddm_utils2d import create_cellsOfFace as cpu_function
from manapy.cuda.manapy.ddm.cuda_ddm_utils2d import get_kernel_create_cellsOfFace as gpu_function

numba_fun = domain.backend.compile(cpu_function, echo=True, signature=True)
cuda_fun = gpu_function()

In [ ]:
numba_fun(*args)
#%timeit numba_fun(*args)

In [ ]:
cuda_fun(*args)
#%timeit cuda_fun(*args)

In [ ]:
# verify

def verify(a, decimal, plot):
  b = VarClass.to_device(a)
  if plot == True:
    plt.plot(a, label="cpu")
    plt.plot(b, label="gpu")
    plt.legend()
    plt.show()
  np.testing.assert_almost_equal(a, b, decimal=decimal)


for i, arg in enumerate(args):
  print(f"{i + 1} => test arg {arg}")
  verify(arg, decimal=2, plot=False)


In [ ]:
dir(np.random)

In [ ]:
import numpy as np

p_faces = (np.random.rand(100).reshape((10, 10)) * 100).astype(np.int32)
res = np.sort(p_faces[:, :-1], axis=1, kind='quicksort')[:, ::-1]
res
#[:, ::-1]

In [ ]:
np.unique(res, axis=0, return_inverse=True)

In [ ]:
from numba import cuda
import numpy as np

a = np.ones(100000000)


@cuda.jit
def test(a: 'int[:]', res: 'int[:]'):
  start = cuda.grid(1)
  grid_size = cuda.gridsize(1)

  if start == 0:
    res[0] = 0
  for idx in range(start, a.shape[0], grid_size):
    cuda.atomic.add(res, -1, a[idx])

d_a = cuda.to_device(a)
d_sum = cuda.to_device(np.zeros(1, dtype=np.int32))
test[1000, 1000](d_a, d_sum)
cuda.synchronize()

In [ ]:
d_sum[0]

In [ ]:
def run():
  test[1000, 1000](d_a, d_sum)
  cuda.synchronize()

%timeit run()

In [ ]:
#    'hex': {'quad': [[0, 1, 2, 3], [0, 1, 4, 5], [1, 2, 5, 6],
#                     [2, 3, 6, 7], [0, 3, 4, 7], [4, 5, 6, 7]]},

import numba
from numba import cuda
import numpy as np

@numba.jit
def create_mesh(nb_x):
  cells = np.zeros(shape=(nb_x, 8), dtype=np.int32)
  cells[0][0] = 0
  cells[0][1] = 1
  cells[0][2] = 2
  cells[0][3] = 3
  for i in range(nb_x):
    cells[i][4] = cells[i][0] + 4
    cells[i][5] = cells[i][1] + 4
    cells[i][6] = cells[i][2] + 4
    cells[i][7] = cells[i][3] + 4

    if i + 1 < nb_x:
      cells[i + 1][0] = cells[i][4]
      cells[i + 1][1] = cells[i][5]
      cells[i + 1][2] = cells[i][6]
      cells[i + 1][3] = cells[i][7]
  return cells



In [ ]:
cell

In [ ]:
nb_cell = 1000000
cell = create_mesh(nb_cell)
neighbors = np.zeros(shape=(nb_cell * 4 + 4, 8 + 1), dtype=np.int32)

@cuda.jit
def create_neighbors_by_vertex(cells: 'int[:, :]', res: 'int[:, :]'):
  start = cuda.grid(1)
  stride = cuda.gridsize(1)
  nb_elements = cells.shape[0]

  for idx in range(start, nb_elements, stride):
    for i in range(cells[idx].shape[0]):
      node = cells[idx][i]
      j = cuda.atomic.add(res[node], -1, 1)
      j = j % 8
      res[node][j] = idx

d_cells = cuda.to_device(cell)
d_neighbors = cuda.to_device(neighbors)

create_neighbors_by_vertex[1000, 1000](d_cells, d_neighbors)

res = d_neighbors.copy_to_host()
res

In [ ]:
def run():
  create_neighbors_by_vertex[1000, 1000](d_cells, d_neighbors)
  cuda.synchronize()

%timeit run()

In [ ]:
res = d_neighbors.copy_to_host()
res

In [ ]:
@cuda.jit(device=True)
def insertion_sort(arr : 'int[:]'):
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and key < arr[j]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key

@cuda.jit
def sort_arr(arr : 'int[:, :]'):
  start = cuda.grid(1)
  stride = cuda.gridsize(1)

  for idx in range(start, arr.shape[0], stride):
    insertion_sort(arr[idx])


In [ ]:
a = np.array(np.random.rand(100000000) * 100, dtype=np.int32)
a = a.reshape(1000000, 100)

d_a = cuda.to_device(a)
sort_arr[1000, 128](d_a)
h_a = d_a.copy_to_host()
h_a

In [ ]:
arr = np.zeros(shape=(1024, 100))
arr[:, -1] = 8
arr

array([[0., 0., 0., ..., 0., 0., 8.],
       [0., 0., 0., ..., 0., 0., 8.],
       [0., 0., 0., ..., 0., 0., 8.],
       ...,
       [0., 0., 0., ..., 0., 0., 8.],
       [0., 0., 0., ..., 0., 0., 8.],
       [0., 0., 0., ..., 0., 0., 8.]])

In [ ]:
arr.shape[0]

1024

In [ ]:
from numba import cuda
import numpy as np

@cuda.jit
def test(arr):
  start = cuda.grid(1)
  stride = cuda.gridsize(1)

  for idx in range(start, 102400, stride):
    for i in range(1, arr[idx][0] + 1):
      arr[idx][i] = 1

size = 64
arr = np.zeros(shape=(102400, size + 1), dtype=np.int32)
arr[:, 0] = size
arr[:, -1] = size

d_arr = cuda.to_device(arr)
test[3200, 32](d_arr)
d_arr[0]

In [ ]:
%timeit test[32, 32](d_arr) ;cuda.synchronize()

/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


3 ms ± 32.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
import numpy as np
from numba import cuda
from numba import jit

#@jit(nopython=True)
def insertion_sort(arr : 'int[:]'):
  for i in range(1, len(arr)):
    key = arr[i]
    j = i - 1
    while j >= 0 and key < arr[j]:
      arr[j + 1] = arr[j]
      j -= 1
    arr[j + 1] = key

#@jit(nopython=True)
def _is_in_array(array: 'int[:]', item : 'int') -> 'int':
  """
    check if item is in array
    return 1 if item in array otherwise 0

    Note:
      the number of item in the array is array[-1]
  """
  for i in range(array[-1]):
    if item == array[i]:
      return 1
  return 0

#@jit(nopython=True)
def _intersect_nodes(face_nodes : 'int[:]', nb_nodes : 'int', neighbors_by_node : 'int[:, :]', intersect_cell : 'int[:]'):
  """
    Get the intersection cells of the face nodes

    Args:
      face_nodes: nodes of the face
      nb_nodes : number of nodes of the face
      neighbors_by_node: for each node get the neighbor cells

    Results:
      intersect_cell: common cells between all neighbors of each node (two at most)
  """
  index = 0

  intersect_cell[0] = -1
  intersect_cell[1] = -1
  
  cells = neighbors_by_node[face_nodes[0]]
  for i in range(cells[-1]):
    intersect_cell[index] = cells[i]
    for j in range(1, nb_nodes):
      if _is_in_array(neighbors_by_node[face_nodes[j]], cells[i]) == 0:
        intersect_cell[index] = -1
        break
    if intersect_cell[index] != -1:
      index = index + 1
    if index >= 2:
      return

#@jit(nopython=True)
def _create_faces(nodes : 'int[:]', out_faces: 'int[:, :]', size_info: 'int[:]', cell_type : 'int'):
  """
    create cell faces

    Args:
      nodes : nodes of the cell
      cell_type :
        # 4 => tetra
        # 8 => hexahedron
        # 5 => pyramid
    
    Return:
      out_faces: faces of the cell
      size_info: 
        contains number of nodes of each face
        and the total number of faces of the cell

    Used Map:
    'tet': {'tri': [[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]]},

    'hex': {'quad': [[0, 1, 2, 3], [0, 1, 4, 5], [1, 2, 5, 6],
                     [2, 3, 6, 7], [0, 3, 4, 7], [4, 5, 6, 7]]},

    'pri': {'quad': [[0, 1, 3, 4], [1, 2, 4, 5], [0, 2, 3, 5]],
            'tri': [[0, 1, 2], [3, 4, 5]]},

    'pyr': {'quad': [[0, 1, 2, 3]],
            'tri': [[0, 1, 4], [1, 2, 4], [2, 3, 4], [0, 3, 4]]}

  """


  if cell_type == 4:
    out_faces[0][0] = nodes[0]
    out_faces[0][1] = nodes[1]
    out_faces[0][2] = nodes[2]
    size_info[0] = 3 #number of nodes

    out_faces[1][0] = nodes[0]
    out_faces[1][1] = nodes[1]
    out_faces[1][2] = nodes[3]
    size_info[1] = 3

    out_faces[2][0] = nodes[0]
    out_faces[2][1] = nodes[2]
    out_faces[2][2] = nodes[3]
    size_info[2] = 3

    out_faces[3][0] = nodes[1]
    out_faces[3][1] = nodes[2]
    out_faces[3][2] = nodes[3]
    size_info[3] = 3
    
    size_info[-1] = 4 #number of faces

  if cell_type == 8:
    out_faces[0][0] = nodes[0]
    out_faces[0][1] = nodes[1]
    out_faces[0][2] = nodes[2]
    out_faces[0][3] = nodes[3]
    size_info[0] = 4

    out_faces[1][0] = nodes[0]
    out_faces[1][1] = nodes[1]
    out_faces[1][2] = nodes[4]
    out_faces[1][3] = nodes[5]
    size_info[1] = 4

    out_faces[2][0] = nodes[1]
    out_faces[2][1] = nodes[2]
    out_faces[2][2] = nodes[5]
    out_faces[2][3] = nodes[6]
    size_info[2] = 4

    out_faces[3][0] = nodes[2]
    out_faces[3][1] = nodes[3]
    out_faces[3][2] = nodes[6]
    out_faces[3][3] = nodes[7]
    size_info[3] = 4

    out_faces[4][0] = nodes[0]
    out_faces[4][1] = nodes[3]
    out_faces[4][2] = nodes[4]
    out_faces[4][3] = nodes[7]
    size_info[4] = 4

    out_faces[5][0] = nodes[4]
    out_faces[5][1] = nodes[5]
    out_faces[5][2] = nodes[6]
    out_faces[5][3] = nodes[7]
    size_info[5] = 4

    size_info[-1] = 6
  
  if cell_type == 5:
    out_faces[0][0] = nodes[0]
    out_faces[0][1] = nodes[1]
    out_faces[0][2] = nodes[2]
    out_faces[0][3] = nodes[3]
    size_info[0] = 4

    out_faces[1][0] = nodes[0]
    out_faces[1][1] = nodes[1]
    out_faces[1][2] = nodes[4]
    size_info[1] = 3

    out_faces[2][0] = nodes[1]
    out_faces[2][1] = nodes[2]
    out_faces[2][2] = nodes[4]
    size_info[2] = 3

    out_faces[3][0] = nodes[2]
    out_faces[3][1] = nodes[3]
    out_faces[3][2] = nodes[4]
    size_info[3] = 3

    out_faces[4][0] = nodes[0]
    out_faces[4][1] = nodes[3]
    out_faces[4][2] = nodes[4]
    size_info[4] = 3

    size_info[-1] = 5

#@jit(nopython=True)
def _has_the_same_items(arr1 : 'int[:]', arr2 : 'int[:]', size : 'int'):
  """
    check if all items of arr1 are in arr2
  """
  for i in range(size):
    found = 0
    for j in range(size):
      if arr1[j] == arr2[i]:
        found = 1
        break
    if found == 0:
      return 0
  return 1

#@jit(nopython=True)
def _has_face(cell_faces : 'int[:]', face : 'int[:]', face_size : 'int', faces : 'int[:, :]'):
  """
    - check `face` is in `cell_faces`
  """
  for i in range(cell_faces[-1]):
    face_nodes = faces[cell_faces[i]]
    if face_size == face_nodes[-1] and _has_the_same_items(face_nodes, face, face_size):
      return cell_faces[i]
  return -1

#@jit(nopython=True)
def create_info(
  cells : 'int[:, :]',
  neighbors_by_node : 'int[:, :]',
  faces : 'int[:, :]',
  cell_faces : 'int[:, :]',
  neighbors_by_face : 'int[:, :]',
  faces_counter : 'int[:]',
  tmp_cell_faces : 'int[:,:]',
  tmp_size_info : 'int[:]'
  ):
  """
    - Create faces
    - Create cells with their corresponding faces. 
    - Create cell neighbors of each face.
  """

  intersect_cells = np.zeros(2, dtype=np.int32)

  for i in range(cells.shape[0]):
    _create_faces(cells[i], tmp_cell_faces, tmp_size_info, cells[i][-1])

    # For every face of cell[j]
    # Get the intersection of this face's nodes' neighboring cells.
    # The result should be two cells `intersect_cells`
    for j in range(tmp_size_info[-1]):
      _intersect_nodes(tmp_cell_faces[j], tmp_size_info[j], neighbors_by_node, intersect_cells)

      # check if the face already created
      face_id = _has_face(cell_faces[i], tmp_cell_faces[j], tmp_size_info[j], faces)
      
      # Create face if note exist
      if face_id == -1:
        face_id = faces_counter[0]
        faces_counter[0] += 1
        # copy nodes from tmp_cell_faces
        for k in range(tmp_size_info[j]):
          faces[face_id][k] = tmp_cell_faces[j][k]
        faces[face_id][-1] = tmp_size_info[j]
      

      # Create cells with their corresponding faces.
      # The face has at most two neighbors
      # Assign this face to both of them
      the_cell = cell_faces[intersect_cells[0]]
      if _is_in_array(the_cell, face_id) == 0:
        tmp_size = the_cell[-1]
        the_cell[tmp_size] = face_id
        the_cell[-1] += 1
      if intersect_cells[1] != -1 and _is_in_array(cell_faces[intersect_cells[1]], face_id) == 0:
        the_cell = cell_faces[intersect_cells[1]]
        tmp_size = the_cell[-1]
        the_cell[tmp_size] = face_id
        the_cell[-1] += 1
      
      # Create cell neighbors of each face
      # face neighbor cells
      neighbors_by_face[face_id][0] = intersect_cells[0]
      neighbors_by_face[face_id][1] = intersect_cells[1]
      
      

def create_neighbors_by_node(cells : 'int[:, :]', neighbors_by_node : 'int[:, :]', nodes_counter : 'int[:]'):
  """
    Create neighbor cells for each node

    Args:
      cells: cells with their nodes
    
    Return:
      neighbors_by_node
      nodes_counter
  """
  nb_cells = cells.shape[0]

  for i in range(nb_cells):
    for j in range(cells[i][-1]):
      node = neighbors_by_node[cells[i][j]]
      size = node[-1]
      node[-1] += 1
      node[size] = i
      if size == 0:
        nodes_counter[0] += 1


#@jit(nopython=True)
def create_cells(nb_x : 'int'):
  cells = np.zeros(shape=(nb_x, 9), dtype=np.int32)
  cells[0][0] = 0
  cells[0][1] = 1
  cells[0][2] = 2
  cells[0][3] = 3
  cells[0][-1] = 8
  for i in range(nb_x):
    cells[i][4] = cells[i][0] + 4
    cells[i][5] = cells[i][1] + 4
    cells[i][6] = cells[i][2] + 4
    cells[i][7] = cells[i][3] + 4

    if i + 1 < nb_x:
      cells[i + 1][0] = cells[i][4]
      cells[i + 1][1] = cells[i][5]
      cells[i + 1][2] = cells[i][6]
      cells[i + 1][3] = cells[i][7]
    cells[i][-1] = 8
  return cells



# Try local
# Try with sort


In [72]:
cells = create_cells(2)
nb_cells = cells.shape[0]
max_nb_faces = 6
max_nb_nodes = 4
max_nb_neighbors_by_node = 8
max_nb_node_by_cell = 8
nb_faces = nb_cells * max_nb_faces
nb_nodes = nb_cells * max_nb_node_by_cell

cell_faces = np.zeros(shape=(nb_cells, max_nb_faces + 1), dtype=np.int32)
neighbors_by_face = np.zeros(shape=(nb_faces, 2), dtype=np.int32)

faces = np.zeros(shape=(nb_faces, max_nb_nodes + 1), dtype=np.int32)
faces_counter = np.zeros(1, dtype=np.int32)

neighbors_by_node = np.zeros(shape=(nb_nodes, max_nb_neighbors_by_node + 1), dtype=np.int32)
nodes_counter = np.zeros(1, dtype=np.int32)

tmp_cell_faces = np.zeros(shape=(max_nb_faces, max_nb_nodes), dtype=np.int32)
tmp_size_info = np.zeros(shape=(max_nb_faces + 1), dtype=np.int32)

intersect_cells = np.zeros(2, dtype=np.int32)

create_neighbors_by_node(cells, neighbors_by_node, nodes_counter)
create_info(cells, neighbors_by_node, faces, cell_faces, neighbors_by_face, faces_counter, tmp_cell_faces, tmp_size_info)

In [73]:
cells

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 4,  5,  6,  7,  8,  9, 10, 11,  8]], dtype=int32)

In [74]:
faces, faces_counter

(array([[ 0,  1,  2,  3,  4],
        [ 0,  1,  4,  5,  4],
        [ 1,  2,  5,  6,  4],
        [ 2,  3,  6,  7,  4],
        [ 0,  3,  4,  7,  4],
        [ 4,  5,  6,  7,  4],
        [ 4,  5,  8,  9,  4],
        [ 5,  6,  9, 10,  4],
        [ 6,  7, 10, 11,  4],
        [ 4,  7,  8, 11,  4],
        [ 8,  9, 10, 11,  4],
        [ 0,  0,  0,  0,  0]], dtype=int32),
 array([11], dtype=int32))

In [75]:
cell_faces

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 5,  6,  7,  8,  9, 10,  6]], dtype=int32)

In [76]:
for item in cell_faces[0]:
  print(faces[item])

[0 1 2 3 4]
[0 1 4 5 4]
[1 2 5 6 4]
[2 3 6 7 4]
[0 3 4 7 4]
[4 5 6 7 4]
[4 5 8 9 4]


In [41]:
_create_faces(cells[0], tmp_cell_faces, tmp_size_info, 8)


_intersect_nodes(tmp_cell_faces[5], tmp_size_info[5], neighbors_by_node, intersect_cells)


In [42]:
intersect_cells

array([0, 1], dtype=int32)

In [ ]:
tmp_size_info

array([4, 4, 4, 4, 4, 4, 6], dtype=int32)

In [98]:
a = np.ones(shape=(10, 10), dtype=np.int32)
a[5][0] = -1
a = a[:6]
a

array([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [-1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int32)

In [108]:
from manapy.ddm.pure_ddm   import read_mesh_file

path = "meshes1PROC"
tc, \
nodeid, \
vertex, \
halosint, \
halosext, \
centvol, \
globtoloc, \
loctoglob, \
loctoglob, \
neigh, \
nparts = read_mesh_file(1, 0, path, float)

In [121]:
a = np.arange(1, 100)
a.resize(10, 10)
print(a[:, -1][:])

[10 20 30 40 50 60 70 80 90  0]
